In [43]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import itertools as itr
import h5py
import os
from functions import *

In [9]:
data_file_name = "SSH_model_v3.h5"
data_file_output_directory = "data"
os.makedirs(data_file_output_directory, exist_ok=True)
data_file_full_path = os.path.join(data_file_output_directory, data_file_name)

file = h5py.File(data_file_full_path, 'a')
grp1 = file["run-1"]

In [10]:
v_arr = np.array(grp1['v_arr'][:])
e_val_arr = np.array(grp1['E_arr'][:])
e_vec_arr = np.array(grp1['E_vec_arr'][:])

In [11]:
print(v_arr)
print('----------')
print(e_val_arr)
print('----------')
print(e_vec_arr)

[0.   0.03 0.06 0.09 0.12 0.15 0.18 0.21 0.24 0.27 0.3  0.33 0.36 0.39
 0.42 0.45 0.48 0.51 0.54 0.57 0.6  0.63 0.66 0.69 0.72 0.75 0.78 0.81
 0.84 0.87 0.9  0.93 0.96 0.99 1.02 1.05 1.08 1.11 1.14 1.17 1.2  1.23
 1.26 1.29 1.32 1.35 1.38 1.41 1.44 1.47 1.5  1.53 1.56 1.59 1.62 1.65
 1.68 1.71 1.74 1.77 1.8  1.83 1.86 1.89 1.92 1.95 1.98 2.01 2.04 2.07
 2.1  2.13 2.16 2.19 2.22 2.25 2.28 2.31 2.34 2.37 2.4  2.43 2.46 2.49
 2.52 2.55 2.58 2.61 2.64 2.67 2.7  2.73 2.76 2.79 2.82 2.85 2.88 2.91
 2.94 2.97 3.  ]
----------
[[ -924.          -924.          -924.         ...   924.
    924.         10428.        ]
 [ -949.07215085  -941.61156804  -930.67195553 ...   932.0772315
    941.6022618  10783.89519575]
 [ -974.32630792  -959.84724759  -938.33235    ...   941.08453343
    959.81301668 11140.08073588]
 ...
 [-3584.22961857 -3420.98778039 -3161.12809709 ...  3195.13244475
   3419.76964691 45640.95177914]
 [-3611.83491147 -3448.27911378 -3188.02465819 ...  3222.0907499
   3447.06566699 4

In [12]:
e_val_arr_abs = np.abs(e_val_arr)

In [24]:
tol = 10**(-17)
topo_state_arr_idx_arr = []
steps = len(e_val_arr)
dim = len(e_val_arr[0])
# print(steps, dim)

for i in range(steps):
    topo_state_arr_idx_arr.append(np.argmin(e_val_arr_abs[i]))

print(len(topo_state_arr_idx_arr))
print(topo_state_arr_idx_arr)

101
[np.int64(1839), np.int64(1846), np.int64(1850), np.int64(1854), np.int64(1845), np.int64(1835), np.int64(1839), np.int64(1840), np.int64(1828), np.int64(1853), np.int64(1849), np.int64(1846), np.int64(1848), np.int64(1843), np.int64(1831), np.int64(1836), np.int64(1833), np.int64(1842), np.int64(1840), np.int64(1838), np.int64(1841), np.int64(1846), np.int64(1851), np.int64(1851), np.int64(1840), np.int64(1846), np.int64(1834), np.int64(1850), np.int64(1843), np.int64(1839), np.int64(1835), np.int64(1849), np.int64(1843), np.int64(1834), np.int64(1835), np.int64(1842), np.int64(1848), np.int64(1848), np.int64(1835), np.int64(1841), np.int64(1849), np.int64(1844), np.int64(1852), np.int64(1841), np.int64(1840), np.int64(1839), np.int64(1842), np.int64(1843), np.int64(1853), np.int64(1843), np.int64(1850), np.int64(1847), np.int64(1837), np.int64(1843), np.int64(1851), np.int64(1841), np.int64(1844), np.int64(1832), np.int64(1845), np.int64(1845), np.int64(1835), np.int64(1854), np.

In [ ]:
topo_state_arr = []

for i in range(steps):
    topo_state_arr.append(e_vec_arr[i][:, topo_state_arr_idx_arr[i]])

topo_state_arr = np.array(topo_state_arr)

In [45]:
def basis_generator_n_2_e(N): # Checked OK

  basis = []

  for combination_indices in itertools.combinations(range(N), N // 2):
    state = [0] * N
    for index in combination_indices:
      state[index] = 1
    basis.append(state)
  
  return np.array(basis, dtype=np.float64)

In [46]:
basis = basis_generator_n_2_e(14)

In [32]:
print(np.shape(topo_state_arr))

(101, 3432)


In [47]:
topo_state_ps_arr = np.zeros((steps, 14))

for i in range(steps):
    for j in range(dim):
        topo_state_ps_arr[i] += topo_state_arr[i][j] * basis[j]

In [48]:
print(np.shape(topo_state_ps_arr))

(101, 14)


In [ ]:
sites_arr = np.arange(14)

for i in range(steps):

    print(np.linalg.norm(topo_state_ps_arr[i]))

for i in range(steps):

    plt.plot(sites_arr, topo_state_ps_arr[i] / np.linalg.norm(topo_state_ps_arr[i]), label=f"{v_arr[i]}")
    plt.legend()
    plt.grid()
    plt.show()

In [54]:
file.close()